<a href="https://colab.research.google.com/github/LexDF/votkai/blob/main/HunyVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #@title Environment Setup

#@markdown ---
#@markdown # Civitai API Keyの設定(HunyuanVideo LoRAのダウンロードに必要です)
#@markdown Civitaiにホストされているモデルをダウンロードする場合はAPIキーが必要になりました。APIキーはCivitaiのサイトにログインした状態で以下のリンクから取得できます。

#@markdown https://civitai.com/user/account

#@markdown **なおCivitaiのAPI Keyは人と共有しないようにお気をつけください**

civitai_api_key = "d9c3085632fb8f24c8561b7068bb3b7f" #@param {type:"string"}

#@markdown ---

### setup ComfyUI ###
from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

### setup  ###
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors -O models/diffusion_models/hunyuan_video_t2v_720p_bf16.safetensors
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/clip_l.safetensors -O models/text_encoders/clip_l.safetensors
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/text_encoders/llava_llama3_fp8_scaled.safetensors -O models/text_encoders/llava_llama3_fp8_scaled.safetensors
!wget https://huggingface.co/Comfy-Org/HunyuanVideo_repackaged/resolve/main/split_files/vae/hunyuan_video_vae_bf16.safetensors -O models/vae/hunyuan_video_vae_bf16.safetensors


#@markdown ---
#@markdown ## LoRA

#@markdown ### Pixel style hunyuan video model
#@markdown [公式ページ](https://civitai.com/models/1144459/pixel-style-hunyuan-video-model?modelVersionId=1287140)
use_pixel_style_hunyuan_video_model = False #@param {type: "boolean"}
if use_pixel_style_hunyuan_video_model:
  !wget https://civitai.com/api/download/models/1287140?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### Studio Ghibli Style 🎞️ HunyuanVideo
#@markdown [公式ページ](https://civitai.com/models/1084814/studio-ghibli-style-hunyuanvideo?modelVersionId=1218122)
use_studio_ghibli_style_hunyuan_video = False #@param {type: "boolean"}
if use_studio_ghibli_style_hunyuan_video:
  !wget https://civitai.com/api/download/models/1218122?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### Flat Color - Style
#@markdown [公式ページ](https://civitai.com/models/1132089/flat-color-style?modelVersionId=1315010)
use_flat_color_style = False #@param {type: "boolean"}
if use_flat_color_style:
  !wget https://civitai.com/api/download/models/1315010?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### HunyuanVideo - Frost Bloom 霜华
#@markdown [公式ページ](https://civitai.com/models/1068416/hunyuanvideo-frost-bloom?modelVersionId=1199182)
use_hunyuan_video_frost_bloom = False #@param {type: "boolean"}
if use_hunyuan_video_frost_bloom:
  !wget https://civitai.com/api/download/models/1199182?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### Minrill Style Animation - Hunyuan Video LoRA
#@markdown [公式ページ](https://civitai.com/models/1120207/minrill-style-animation-hunyuan-video-lora?modelVersionId=1258988)
use_minrill_style_animation_hunyuan_video_lora = False #@param {type: "boolean"}
if use_minrill_style_animation_hunyuan_video_lora:
  !wget https://civitai.com/api/download/models/1258988?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### underground club
#@markdown [公式ページ](https://civitai.com/models/314278/underground-club?modelVersionId=1237165)
use_underground_club = False #@param {type: "boolean"}
if use_underground_club:
  !wget https://civitai.com/api/download/models/1237165?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

#@markdown ### CoolGuyExplosion
#@markdown [公式ページ](https://civitai.com/models/1100877/coolguyexplosion?modelVersionId=1236617)
use_cool_guy_explosion = False #@param {type: "boolean"}
if use_cool_guy_explosion:
  !wget https://civitai.com/api/download/models/1236617?token={civitai_api_key} --content-disposition --directory-prefix=/content/ComfyUI/models/loras/

### start comfyui ###
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server